In [1]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)


2023-01-20 00:11:18 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75


In [4]:
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(models, cfg)

text = "o knee chan" #cursed

model = model.to("cuda:0")

sample = TTSHubInterface.get_model_input(task, text)
sample["net_input"]["src_tokens"] = sample["net_input"]["src_tokens"].to("cuda:0")
sample["net_input"]["src_lengths"] = sample["net_input"]["src_lengths"].to("cuda:0")
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)


2023-01-20 00:12:13 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from C:/Users/ryanr/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


In [10]:
import numpy as np
from scipy.io.wavfile import write

data = wav.cpu().numpy() 
scaled = np.int16(data/np.max(np.abs(data)) * 32767)
write('test.wav', int(rate*1.4), scaled)